In [1]:
import snowflake.connector
import pandas as pd

In [2]:
ctx = snowflake.connector.connect(
    user='UCF_GROUP_2',
    password='UCFmsda2019$',
    account='qaa09892.us-east-1'
    )
ctx.cursor().execute('USE DATABASE UCF_METER_DATA')
ctx.cursor().execute('USE UCF_METER_DATA.DWH')

In [3]:

query = '''
SELECT a.METER_SCHEDULE,
a.IS_BILLING_FLG,
a.COLLECTOR_NAME,
a.COLLECTION_TS_LTZ,
a.METER_NAME as meternamei,
a.INTERVAL_SECONDS,
a.NUM_INTERVAL,
a.SUM_INTERVALS,
a.UOM,
a.DIRECTION,
a.CHANNEL,
a.MULTIPLIER,
a.READING_TS_LTZ,
a.READING,
a.DATETIME_KEY as DATETIME_KEYi,
a.COLLECTOR_KEY,
b.METER_NAME as  meternamed,
b.MR_CYC_CD,
b.SEQ,
b.SP_ID,
b.RPS_AREA,
b.LATITUDE,
b.LONGITUDE,
b.TRNSFRMR,
b.WATER_ONLY_FLG,
b.MODEL_CD,
b.LOCATION,
b.ROUTE_ID,
b.INSTALLED_REMOVED,
b.IS_CURRENT_FLG,
c.METER_NAME as meternames,
c.STATUS_CATEGORY,
c.datetime_key as DATETIME_KEYs

 FROM FACT_INTERVAL a  inner join DIM_METER_SYNC b on a.METER_NAME=b.METER_NAME left join FACT_STATUS c on  a.METER_NAME=c.METER_NAME and a.DATETIME_KEY=c.DATETIME_KEY  and c.STATUS_CATEGORY='Tamper Alert' 
 WHERE  a.METER_NAME IN (select DISTINCT  METER_NAME from DIM_METER_SYNC sample(1) limit 100) and a.direction ='Delivered' and b.INSTALLED_REMOVED='I'
'''





In [4]:
cur = ctx.cursor().execute(query)
df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [10]:
df.to_csv(r'C:\Users\kbvip\Desktop\OUC\oucdata.csv')

In [15]:
df=df.drop(columns=['METERNAMES','DATETIME_KEYS','METERNAMED']) 

In [17]:
df['STATUS_CATEGORY']=df['STATUS_CATEGORY'].fillna(0)

In [20]:
df['STATUS_CATEGORY']=df['STATUS_CATEGORY'].replace('Tamper Alert',1)

In [22]:
df.to_csv(r'C:\Users\kbvip\Desktop\OUC\oucdata1.csv')

In [24]:
df.head()

,METER_SCHEDULE,IS_BILLING_FLG,COLLECTOR_NAME,COLLECTION_TS_LTZ,METERNAMEI,INTERVAL_SECONDS,NUM_INTERVAL,SUM_INTERVALS,UOM,DIRECTION,...,LATITUDE,LONGITUDE,TRNSFRMR,WATER_ONLY_FLG,MODEL_CD,LOCATION,ROUTE_ID,INSTALLED_REMOVED,IS_CURRENT_FLG,STATUS_CATEGORY
0,LP_General,False,NMC00084,2020-01-13 01:15:00-05:00,6CD61776,900,84,40.7280,kWh,Delivered,...,28.2493,-81.2343,82854,False,R2SD,None,00000021,I,True,0
1,LP_General,False,NMC00084,2020-01-13 01:15:00-05:00,6CD61776,900,84,40.7280,kWh,Delivered,...,28.2493,-81.2343,82854,False,R2SD,None,00000021,I,True,0
2,LP_General,False,NMC00084,2020-01-14 01:15:00-05:00,6CD61776,900,84,38.0940,kWh,Delivered,...,28.2493,-81.2343,82854,False,R2SD,None,00000021,I,True,0
3,LP_General,False,NMC00084,2020-01-14 01:15:00-05:00,6CD61776,900,84,38.0940,kWh,Delivered,...,28.2493,-81.2343,82854,False,R2SD,None,00000021,I,True,0
4,LP_General,False,NMC00084,2020-01-14 01:15:00-05:00,6CD61776,900,84,38.0940,kWh,Delivered,...,28.2493,-81.2343,82854,False,R2SD,None,00000021,I,True,0
